# Testing hexagonal kernels and backward gradients

In [6]:
import os
import sys
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.logging.set_verbosity(tf.logging.DEBUG)
sys.path.append(os.path.abspath('../'))

In [7]:
ls

BUILD         eval.py                parameters.py         train.py
checkpoints/  inspect_checkpoint.py  preprocessing/        WORKSPACE
datasets/     LICENSE                README.md
deploy/       models/                tf_cnn_benchmarks.py
docs/         notebooks/             tf_extended/


## Hexagonal layers

Test out the basic hexagonal layers.

In [10]:
# from tensorflow.contrib import hex_layers
import tf_extended as tfx
import tf_extended as tfx
from tensorflow.contrib import hex_layers

ImportError: cannot import name 'hex_layers'

In [ ]:
size = 6
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

In [ ]:
t = hex_layers.hex_depthwise_conv2d(a, 3)
t = hex_layers.hex_downscale2d(a, 2)

In [ ]:
# Test cartesian to hexagonal tiling.
t = hex_layers.hex_from_cartesian(a, downscale=False)
with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    print(t.eval().squeeze())



In [ ]:
a.shape
t.get_shape()


In [ ]:
aa = tf.convert_to_tensor(a)
l0 = aa[:,0]
print(l0.get_shape())

# aa.

In [ ]:
5 % 3

## Forward hexagonal depthwise convolution 2D

Testing the forward convolution.

In [ ]:
size = 5
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

In [ ]:
# Testing several simple configurations of kernels.
radius = 2
ksize = 2*radius+1
stride = 1

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    idx = 0
    for rad in range(radius+1):
        rsize = max(1, rad*6)
        for i in range(rsize):
            # Basic kernel
            f = np.zeros(shape=[ksize**2], dtype=np.float32)
            f[idx] = 1.
            print('TEST kernel %i: \n' % idx, f)
            # Inverse kernel.
            delta = rad*3 if i < rad*3 else -rad*3
            fi = np.zeros(shape=[ksize**2], dtype=np.float32)
            fi[idx+delta] = 1.
            # Hex. depthwise conv2d
            f = np.reshape(f, [ksize, ksize, 1, 1])
            t = tf.nn.hex_depthwise_conv2d(a, f, [1, stride, stride, 1], 'SAME', data_format='NHWC')
            r = t.eval()
            # Inverse computation
            fi = np.reshape(fi, [ksize, ksize, 1, 1])
            ti = tf.nn.hex_depthwise_conv2d(r, fi, [1, stride, stride, 1], 'SAME', data_format='NHWC')
            ri = ti.eval()
            
            print('TEST result: \n', r.squeeze(), '\nInverse:\n', ri.squeeze(), '\n')
            idx += 1

#     for k in range(ksize**2):
#         # Basic kernel
#         f = np.zeros(shape=[ksize**2], dtype=np.float32)
#         f[k] = 1.
#         print('TEST kernel %i: \n' % k, f)
#         # Hex. depthwise conv2d
#         f = np.reshape(f, [ksize, ksize, 1, 1])
#         t = tf.nn.hex_depthwise_conv2d(a, f, [1, stride, stride, 1], 'SAME', data_format='NHWC')
#         r = t.eval()
#         print('TEST result: \n', r.squeeze(), '\n')


## Backward hexagonal depthwise convolution 2D - Input

Testing the backward convolution, input gradient.

In [ ]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

In [ ]:
# Testing several simple configurations of kernels.
insize = 5
ksize = 3
stride = 2

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST kernel: \n', f)
        # Hex. depthwise conv2d
        f = np.reshape(f, [ksize, ksize, 1, 1])
        t = tf.nn.hex_depthwise_conv2d_native_backprop_input(
            [1, insize, insize, 1], f, a, [1, stride, stride, 1], 'SAME', data_format='NHWC')        
        print('TEST result: \n', t.eval().squeeze(), '\n')



## Backward hexagonal depthwise convolution 2D - kernel

Testing the backward convolution, input gradient.

In [ ]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

In [ ]:
# Testing several simple configurations of kernels.
ksize = 3

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST out gradient: \n', f.reshape([3, 3]))
        # Hex. depthwise conv2d
        f = np.reshape(f, [1, ksize, ksize, 1])
        t = tf.nn.hex_depthwise_conv2d_native_backprop_filter(
            f, [3, 3, 1, 1], a, [1, 1, 1, 1], 'SAME', data_format='NHWC')        
        print('TEST result: \n', t.eval().squeeze().flatten(), '\n')


## Help...

In [ ]:
tf.nn.hex_depthwise_conv2d?

In [ ]:
tf.nn.depthwise_conv2d_native_backprop_filter?

In [ ]:
tf.nn.hex_depthwise_conv2d_native_backprop_input?

In [ ]:
tf.nn.hex_depthwise_conv2d_native_backprop_filter?